In [2]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

In [14]:
def apply_dipole_interaction(circuit, qubits, chi_t):
    n = len(qubits)
    for i in range(n):
        for j in range(i + 1, n):
            qi, qj = qubits[i], qubits[j]

            # ZZ term
            circuit.append(RZZGate(2 * chi_t), [qi, qj])

            # XX term 
            theta_x = -chi_t
            circuit.h(qi); circuit.h(qj)
            circuit.cx(qi, qj)
            circuit.rz(theta_x, qj)
            circuit.cx(qi, qj)
            circuit.h(qi); circuit.h(qj)

            # YY term
            theta_y = -chi_t
            circuit.rx(np.pi/2, qi); circuit.rx(np.pi/2, qj)
            circuit.cx(qi, qj)
            circuit.rz(theta_y, qj)
            circuit.cx(qi, qj)
            circuit.rx(-np.pi/2, qi); circuit.rx(-np.pi/2, qj)
            
    return circuit

In [12]:
def calculate_jz_and_delta_jz(counts, n_qubits=3):
    expectation = 0
    expectation_sq = 0
    total_shots = sum(counts.values())
    
    for bitstring, count in counts.items():
        n_zeros = bitstring.count('0')  
        n_ones = bitstring.count('1')   
        jz_value = (n_zeros - n_ones) / 2
        prob = count / total_shots
        expectation += prob * jz_value
        expectation_sq += prob * (jz_value ** 2)
    
    delta_jz = (expectation_sq - expectation ** 2) ** 0.5
    return expectation, delta_jz

In [10]:
# Parameters
N_values = range(2, 9)              
tau_ns_max = 650.0                    
tau_points = 60
tau_grid_s = np.linspace(0.0, tau_ns_max * 1e-9, tau_points)  
shots = 1000
backend = AerSimulator()

In [4]:
d_closed_hz  = 1e6   # d = 1
d_sql_hz     = 0.0   # d = 0 (SQL)

# Basis-rotation functions 
def rotate_for_X_basis(qc, qubits):
    for q in qubits:
        qc.h(q)

def rotate_for_Y_basis(qc, qubits):
    for q in qubits:
        qc.sdg(q); qc.h(q)

In [6]:
def delta_J_component(N, d_hz, tau_s, shots, component: str) -> float:
    chi_t = 2 * np.pi * d_hz * tau_s

    qc = QuantumCircuit(N)
    qc.h(range(N)) 

    # Interaction ON unless d = 0
    if d_hz != 0.0:
        apply_dipole_interaction(qc, list(range(N)), chi_t)   # <- your function

    # Rotate to measurement basis
    comp = component.lower()
    if comp == 'x':
        rotate_for_X_basis(qc, range(N))
    elif comp == 'y':
        rotate_for_Y_basis(qc, range(N))
    elif comp == 'z':
        pass
    else:
        raise ValueError("component must be 'x','y','z'")

    qc.measure_all()
    counts = backend.run(transpile(qc, backend), shots=shots).result().get_counts()
    
    _, dJ = calculate_jz_and_delta_jz(counts, n_qubits=N)  
    return float(dJ)

In [8]:
def sigma_min_curve_normalized(N, d_hz):
    sigma0 = np.sqrt(N) / 2.0
    vals = []
    for tau in tau_grid_s:
        dx = delta_J_component(N, d_hz, tau, shots, 'x')
        dy = delta_J_component(N, d_hz, tau, shots, 'y')
        dz = delta_J_component(N, d_hz, tau, shots, 'z')
        sigma_t = min(dx, dy, dz)
        vals.append(sigma_t / sigma0)
    return np.array(vals)

In [ ]:
# Compute curves for both cases
curves_d0      = {N: sigma_min_curve_normalized(N, d_sql_hz)    for N in N_values}
curves_closed  = {N: sigma_min_curve_normalized(N, d_closed_hz) for N in N_values}

In [ ]:
tau_ns = tau_grid_s * 1e9
fig, axes = plt.subplots(1, 2, figsize=(11, 4.6), sharey=True)

# d = 0
ax = axes[0]
for N in N_values:
    ax.plot(tau_ns, curves_d0[N], label=f"N={N}")
ax.axhline(1.0, ls="--", color="k", alpha=0.6, label="SQL (σ/σ₀=1)")
ax.set_title("d = 0")
ax.set_xlabel("Evolution time τ (ns)")
ax.set_ylabel(r"Normalized uncertainty  $\sigma_{\min}/\sigma_0$")
ax.grid(True, ls="--", alpha=0.4)

# d = 1
ax = axes[1]
for N in N_values:
    ax.plot(tau_ns, curves_closed[N], label=f"N={N}")
ax.axhline(1.0, ls="--", color="k", alpha=0.6, label="SQL (σ/σ₀=1)")
ax.set_title(r"$d/(2\pi) = 1\ \mathrm{MHz}$")
ax.set_xlabel("Evolution time τ (ns)")
ax.grid(True, ls="--", alpha=0.4)

axes[1].legend(bbox_to_anchor=(1.02, 1.0), loc="upper left", borderaxespad=0.)
plt.tight_layout()
plt.show()